In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import codecs
import json
import base64
from PIL import Image
from io import BytesIO
import os

In [5]:
LAWMAKER_NAME_CLEANING = {
    'Ankney, Duane': 'Duane Ankney',
    'Fitzpatrick, Steve': 'Steve Fitzpatrick',
    'Malek, Sue': 'Sue Malek',
    'Sesso, Jon': 'Jon Sesso',
    'Sesso, Jon C.': 'Jon Sesso',
    'Jon C Sesso': 'Jon Sesso',
    'Barrett, Dick': 'Dick Barrett',
    'Flowers, Pat': 'Pat Flowers',
    'McClafferty, Edith': 'Edie McClafferty',
    'McClafferty, Edith (Edie)': 'Edie McClafferty',
    'Edith (Edie) McClafferty': 'Edie McClafferty',
    'McClafferty, Edith(Edie)': 'Edie McClafferty',
    'Edie Mcclafferty': 'Edie McClafferty',
    'Small, Jason': 'Jason Small',
    'Small, Jason D.': 'Jason Small',
    'Jason D Small': 'Jason Small',
    'Small, JasonD.': 'Jason Small',
    'Bennett, Bryce': 'Bryce Bennett',
    'Gauthier, Terry': 'Terry Gauthier',
    'McConnell, Nate': 'Nate McConnell',
    'Smith, Cary': 'Cary Smith',
    'Blasdel, Mark': 'Mark Blasdel',
    'Gillespie, Bruce': 'Bruce Gillespie',
    'McNally, Mary': 'Mary McNally',
    'Smith, Frank': 'Frank Smith',
    'Bogner, Kenneth': 'Kenneth Bogner',
    'Gross, Jen': 'Jen Gross',
    'Olszewski, Albert': 'Albert Olszewski',
    'Tempel, Russel': 'Russ Tempel',
    'Tempel, Russel (Russ)': 'Russ Tempel',
    'Russel (Russ) Tempel': 'Russ Tempel',
    'Tempel, Russel(Russ)': 'Russ Tempel',
    'Boland, Carlie': 'Carlie Boland',
    'Hinebauch, Steve': 'Steve Hinebauch',
    'Osmundson, Ryan': 'Ryan Osmundson',
    'Thomas, Fred': 'Fred Thomas',
    'Brown, Dee': 'Dee Brown',
    'Hoven, Brian': 'Brian Hoven',
    'Phillips, Mike': 'Mike Phillips',
    'Vance, Gordon': 'Gordon Vance',
    'Cohenour, Jill': 'Jill Cohenour',
    'Howard, David': 'David Howard',
    'Pomnichowski, JP': 'JP Pomnichowski',
    'Vuckovich, Gene': 'Gene Vuckovich',
    'Cuffe, Mike': 'Mike Cuffe',
    'Jacobson, Tom': 'Tom Jacobson',
    'Regier, Keith': 'Keith Regier',
    'Webb, Roger': 'Roger Webb',
    'Ellis, Janet': 'Janet Ellis',
    'Kary, Douglas': 'Doug Kary',
    'Kary, Douglas (Doug)': 'Doug Kary',
    'Douglas (Doug) Kary': 'Doug Kary',
    'Kary, Douglas(Doug)': 'Doug Kary',
    'Richmond, Tom': 'Tom Richmond',
    'Webber, Susan': 'Susan Webber',
    'Ellsworth, Jason': 'Jason Ellsworth',
    'Ellsworth, Jason W.': 'Jason Ellsworth',
    'Jason W Ellsworth': 'Jason Ellsworth',
    'Ellsworth, JasonW.': 'Jason Ellsworth',
    'Keenan, Bob': 'Bob Keenan',
    'Sales, Scott': 'Scott Sales',
    'Welborn, Jeffery': 'Jeff Welborn',
    'Welborn, Jeffrey W.': 'Jeff Welborn',
    'Jeffery W Welborn': 'Jeff Welborn',
    'Jeffrey W Welborn': 'Jeff Welborn',
    'Welborn, Jeffrey': 'Jeff Welborn',
    'Welborn, JeffreyW.': 'Jeff Welborn',
    'Esp, John': 'John Esp',
    'Lang, Mike': 'Mike Lang',
    'Salomon, Daniel': 'Dan Salomon',
    'Salomon, Daniel R.': 'Dan Salomon',
    'Daniel R Salomon': 'Dan Salomon',
    'Salomon, DanielR.': 'Dan Salomon',
    'Fielder, Jennifer': 'Jennifer Fielder',
    'MacDonald, Margaret': 'Margie MacDonald',
    'MacDonald, Margaret (Margie)': 'Margie MacDonald',
    'Margaret (Margie) MacDonald': 'Margie MacDonald',
    'Sands, Diane': 'Diane Sands',
    'Abbott, Kim': 'Kim Abbott',
    'Farris-Olsen, Robert': 'Robert Farris-Olsen',
    'Keogh, Connie': 'Connie Keogh',
    'Redfield, Alan': 'Alan Redfield',
    'Anderson, Fred': 'Fred Anderson',
    'Fern, Dave': 'Dave Fern',
    'Kerr-Carpenter, Emma': 'Emma Kerr-Carpenter',
    'Emma Kerr': 'Emma Kerr-Carpenter',
    'Regier, Matt': 'Matt Regier',
    'Bachmeier, Jacob': 'Jacob Bachmeier',
    'Fitzgerald, Ross': 'Ross Fitzgerald',
    'Fitzgerald, Ross H.': 'Ross Fitzgerald',
    'Ross H Fitzgerald': 'Ross Fitzgerald',
    'Fitzgerald, RossH.': 'Ross Fitzgerald',
    'Knudsen, Rhonda': 'Rhonda Knudsen',
    'Ricci, Vince': 'Vince Ricci',
    'Bahr, Jade': 'Jade Bahr',
    'Fleming, Frank': 'Frank Fleming',
    'Knudsen, Casey': 'Casey Knudsen',
    'Running Wolf, Tyson': 'Tyson Running Wolf',
    'Runningwolf, Tyson T.': 'Tyson Running Wolf',
    'Tyson T Running Wolf': 'Tyson Running Wolf',
    'Running Wolf, Tyson T.': 'Tyson Running Wolf',
    'RunningWolf, TysonT.': 'Tyson Running Wolf',
    'Ballance, Nancy': 'Nancy Ballance',
    'Fuller, John': 'John Fuller',
    'Krautter, Joel': 'Joel Krautter',
    'Krautter, Joel G.': 'Joel Krautter',
    'Joel G Krautter': 'Joel Krautter',
    'Ryan, Marilyn': 'Marilyn Ryan',
    'Bartel, Dan': 'Dan Bartel',
    'Funk, Moffie': 'Moffie Funk',
    'Krotkov, Jasmine': 'Jasmine Krotkov',
    'Sales, Walt': 'Walt Sales',
    'Beard, Becky': 'Becky Beard',
    'Galt, Wylie': 'Wylie Galt',
    'Lenz, Dennis': 'Dennis Lenz',
    'Lenz, Dennis R.': 'Dennis Lenz',
    'Dennis R Lenz': 'Dennis Lenz',
    'Lenz, DennisR.': 'Dennis Lenz',
    'Schreiner, Casey': 'Casey Schreiner',
    'Bedey, David': 'David Bedey',
    'Garcia, Rodney': 'Rodney Garcia',
    'Loge, Denley': 'Denley Loge',
    'Loge, Denley M.': 'Denley Loge',
    'Denley M Loge': 'Denley Loge',
    'Loge, DenleyM.': 'Denley Loge',
    'Shaw, Ray': 'Ray Shaw',
    'Shaw, Ray L.': 'Ray Shaw',
    'Ray L Shaw': 'Ray Shaw',
    'Berglee, Seth': 'Seth Berglee',
    'Garner, Frank': 'Frank Garner',
    'Lynch, Ryan': 'Ryan Lynch',
    'Sheldon-Galloway, Lola': 'Lola Sheldon-Galloway',
    'Galloway, Lola': 'Lola Sheldon-Galloway',
    'Lola Sheldon': 'Lola Sheldon-Galloway',
    'Bessette, Barbara': 'Barbara Bessette',
    'Glimm, Carl': 'Carl Glimm',
    'Mandeville, Forrest': 'Forrest Mandeville',
    'Skees, Derek': 'Derek Skees',
    'Bishop, Laurie': 'Laurie Bishop',
    'Greef, Sharon': 'Sharon Greef',
    'Manzella, Theresa': 'Theresa Manzella',
    'Smith, Bridget': 'Bridget Smith',
    'Brown, Bob': 'Bob Brown',
    'Grubbs, Bruce': 'Bruce Grubbs',
    'Marler, Marilyn': 'Marilyn Marler',
    'Stewart Peregoy, Sharon': 'Sharon Stewart Peregoy',
    'StewartPeregoy, Sharon': 'Sharon Stewart Peregoy',
    'Brown, Zach': 'Zach Brown',
    'Gunderson, Steve': 'Steve Gunderson',
    'McKamey, Wendy': 'Wendy McKamey',
    'Sullivan, Katie': 'Katie Sullivan',
    'Burnett, Tom': 'Tom Burnett',
    'Hamilton, Jim': 'Jim Hamilton',
    'Mercer, Bill': 'Bill Mercer',
    'Sweeney, Mark': 'Mark Sweeney',
    'Buttrey, Edward': 'Ed Buttrey',
    'Edward Buttrey': 'Ed Buttrey',
    'Hamlett, Bradley': 'Bradley Hamlett',
    'Moore, Frederick': 'Eric Moore',
    'Moore, Frederick (Eric)': 'Eric Moore',
    'Frederick (Eric) Moore': 'Eric Moore',
    'Tschida, Brad': 'Brad Tschida',
    'Caferro, Mary': 'Mary Caferro',
    'Harvey, Derek': 'Derek Harvey',
    'Harvey, Derek J.': 'Derek Harvey',
    'Derek J Harvey': 'Derek Harvey',
    'Harvey, DerekJ.': 'Derek Harvey',
    'Moore, Terry': 'Terry Moore',
    'Usher, Barry': 'Barry Usher',
    'Curdy, Willis': 'Willis Curdy',
    'Hayman, Denise': 'Denise Hayman',
    'Morigeau, Shane': 'Shane Morigeau',
    'Morigeau, Shane A.': 'Shane Morigeau',
    'Shane A Morigeau': 'Shane Morigeau',
    'Morigeau, ShaneA.': 'Shane Morigeau',
    'Vinton, Sue': 'Sue Vinton',
    'Custer, Geraldine': 'Geraldine Custer',
    'Hertz, Greg': 'Greg Hertz',
    'Mortensen, Dale': 'Dale Mortensen',
    'Weatherwax, Marvin': 'Marvin Weatherwax',
    'DeVries, Greg': 'Greg DeVries',
    'Holmlund, Kenneth': 'Kenneth Holmlund',
    'Holmlund, Kenneth L.': 'Kenneth Holmlund',
    'Kenneth L Holmlund': 'Kenneth Holmlund',
    'Holmlund, KennethL.': 'Kenneth Holmlund',
    'Noland, Mark': 'Mark Noland',
    'Webb, Peggy': 'Peggy Webb',
    'Doane, Alan': 'Alan Doane',
    'Hopkins, Mike': 'Mike Hopkins',
    'Olsen, Andrea': 'Andrea Olsen',
    'Welch, Tom': 'Tom Welch',
    'Dooling, Julie': 'Julie Dooling',
    'Jones, Llew': 'Llew Jones',
    'Peppers, Rae': 'Rae Peppers',
    'White, Kerry': 'Kerry White',
    'Dudik, Kimberly': 'Kim Dudik',
    'Kimberly Dudik': 'Kim Dudik',
    'Karjala, Jessica': 'Jessica Karjala',
    'Perry, Zac': 'Zac Perry',
    'Windy Boy, Jonathan': 'Jonathan Windy Boy',
    'WindyBoy, Jonathan': 'Jonathan Windy Boy',
    'Dunn, David': 'David Dunn',
    'Kassmier, Joshua': 'Josh Kassmier',
    'Joshua Kassmier': 'Josh Kassmier',
    'Pierson, Gordon': 'Gordon Pierson',
    'Winter, Thomas': 'Tom Winter',
    'Thomas Winter': 'Tom Winter',
    'Dunwell, Mary Ann': 'Mary Ann Dunwell',
    'Dunwell, MaryAnn': 'Mary Ann Dunwell',
    'Keane, Jim': 'Jim Keane',
    'Pope, Christopher': 'Chris Pope',
    'Christopher Pope': 'Chris Pope',
    'Woods, Tom': 'Tom Woods',
    'Duram, Neil': 'Neil Duram',
    'Kelker, Kathy': 'Kathy Kelker',
    'Read, Joe': 'Joe Read',
    'Zolnikov, Daniel': 'Daniel Zolnikov',

    'Bertoglio, Marta': 'Marta Bertoglio',
    'Binkley, Michele': 'Michele Binkley',
    'Boldman, Ellie': 'Ellie Boldman',
    'Brewster, Larry': 'Larry Brewster',
    'Buckley, Alice': 'Alice Buckley',
    'Carlson, Jennifer': 'Jennifer Carlson',
    'Fielder, Paul': 'Paul Fielder',
    'Fox, Mike': 'Mike Fox',
    'France, Tom': 'Tom France',
    'Frazer, Gregory': 'Gregory Frazer',
    'Friedel, Chris': 'Chris Friedel',
    'Galloway, Steven': 'Steven Galloway',
    'Gillette, Jane': 'Jane Gillette',
    'Gist, Steve': 'Steve Gist',
    'Hawk, Donavon': 'Donavon Hawk',
    'Hill, Ed': 'Ed Hill',
    'Hinkle, Jedediah': 'Jedediah Hinkle',
    'Hinkle, Caleb': 'Caleb Hinkle',
    'Kerns, Scot': 'Scot Kerns',
    'Kortum, Kelly': 'Kelly Kortum',
    'Ler, Brandon': 'Brandon Ler',
    'Malone, Marty': 'Marty Malone',
    'Marshall, Ron': 'Ron Marshall',
    'McGillvray, Tom': 'Tom McGillvray',
    'Mitchell, Braxton': 'Braxton Mitchell',
    'Molnar, Brad': 'Brad Molnar',
    'Nave, Fiona': 'Fiona Nave',
    'Novak, Sara': 'Sara Novak',
    'O\'Brien, Shannon': 'Shannon O\'Brien',
    'Shannon O': 'Shannon O\'Brien',
    'Patelis, Jimmy': 'Jimmy Patelis',
    'Phalen, Bob': 'Bob Phalen',
    'Putnam, Brian': 'Brian Putnam',
    'Regier, Amy': 'Amy Regier',
    'Reksten, Linda': "Linda Reksten",
    '<skip>, Linda': "Linda Reksten",
    'Schillinger, Jerry': "Jerry Schillinger",
    'Seekins-Crowe, Kerri': "Kerri Seekins-Crowe",
    'Kerri Seekins': "Kerri Seekins-Crowe",
    'Stafman, Ed': "Ed Stafman",
    'Stromswold, Mallerie': "Mallerie Stromswold",
    'Tenenbaum, Danny': "Danny Tenenbaum",
    'Thane, Mark': "Mark Thane",
    'Trebas, Jeremy': "Jeremy Trebas",
    'Walsh, Kenneth': "Kenneth Walsh",
    'Whitman, Kathy': 'Kathy Whitman',
    'Whiteman Pena, Rynalea': 'Rynalea Whiteman Pena',
    'WhitemanPena, Rynalea': 'Rynalea Whiteman Pena',
    'Zolnikov, Katie': 'Katie Zolnikov',

    'Brad Barker': 'Brad Barker',
    'Denise Baum': 'Denise Baum',
    'James Bergstrom': 'James Bergstrom',
    'Tony Brockman': 'Tony Brockman',
    'Bob Carter': 'Bob Carter',
    'Robert Carter': 'Bob Carter',
    'Carter, Robert': 'Bob Carter',
    'Lee Deming': 'Lee Deming',
    'Sherry Essmann': 'Sherry Essmann',
    'Jodee Etchart': 'Jodee Etchart',
    'Terry Falk': 'Terry Falk',
    'John Fitzpatrick': 'John Fitzpatrick',
    'Doug Flament': 'Doug Flament',
    'Paul Green': 'Paul Green',
    'Lyn Hellegaard': 'Lyn Hellegaard',
    'Sj Howell': 'SJ Howell',
    'Jonathan Karlen': 'Jonathan Karlen',
    'Greg Kmetz': 'Greg Kmetz',
    'Jennifer Lynch': 'Jennifer Lynch',
    'Eric Matthews': 'Eric Matthews',
    'Russ Miner': 'Russ Miner',
    'Russel Miner': 'Russ Miner',
    'Miner, Russel': 'Russ Miner',
    'Nelly Nicol': 'Nelly Nicol',
    'George Nikolakakos': 'George Nikolakakos',
    'Greg Oblander': 'Greg Oblander',
    'Gary Parry': 'Gary Parry',
    'Melissa Romano': 'Melissa Romano',
    'Wayne Rusk': 'Wayne Rusk',
    'Laura Smith': 'Laura Smith',
    'Tanner Smith': 'Tanner Smith',
    'Courtenay Sprunger': 'Courtenay Sprunger',
    'Paul Tuss': 'Paul Tuss',
    'Mike Yakawich': 'Mike Yakawich',
    'Michael Yakawich': 'Mike Yakawich',
    'Zooey Zephyr': 'Zooey Zephyr',
    'Daniel Emrich': 'Daniel Emrich',
    'Tom Mcgillvray': 'Tom McGillvray',
    'Wendy Mckamey': 'Wendy McKamey',
    'Shelley Vance': 'Shelley Vance',
    'Terry Vermeire': 'Terry Vermeire',
    'Zack Wirth': 'Zack Wirth',
    'Kmetz, Greg': 'Greg Kmetz',
    'Romano, Melissa': 'Melissa Romano',
    'Fitzpatrick, John': 'John Fitzpatrick',
    'Barker, Brad': 'Brad Barker',
    'Flament, Doug': 'Doug Flament',
    'Rusk, Wayne': 'Wayne Rusk',
    'Baum, Denise': 'Denise Baum',
    'Bergstrom, James': 'James Bergstrom',
    'Lynch, Jennifer': 'Jennifer Lynch',
    'Smith, Tanner': 'Tanner Smith',
    'Green, Paul': 'Paul Green',
    'Smith, Laura': 'Laura Smith',
    'Sprunger, Courtenay': 'Courtenay Sprunger',
    'Brockman, Tony': 'Tony Brockman',
    'Matthews, Eric': 'Eric Matthews',
    'Hellegaard, Lyn': 'Lyn Hellegaard',
    'Tuss, Paul': 'Paul Tuss',
    'Nicol, Nelly': 'Nelly Nicol',
    'Deming, Lee': 'Lee Deming',
    'Howell, SJ': 'SJ Howell',
    'Nikolakakos, George': 'George Nikolakakos',
    'Oblander, Greg': 'Greg Oblander',
    'Karlen, Jonathan': 'Jonathan Karlen',
    'Parry, Gary': 'Gary Parry',
    'Essmann, Sherry': 'Sherry Essmann',
    'Etchart, Jodee': 'Jodee Etchart',
    'Wirth, Zack': 'Zack Wirth',
    'Falk, Terry': 'Terry Falk',
    'Yakawich, Michael': 'Mike Yakawich',
    'Michael Yakawich': 'Mike Yakawich',
    'Zephyr, Zooey': 'Zooey Zephyr',
}

In [14]:
def get_name(page):
    title = page.find('h3').text
    name = re.search(r'(?<=(Sen|Rep)\. )(\w| )+', title).group(0)
    name = name.replace('  ',' ').title()
    if name not in LAWMAKER_NAME_CLEANING.keys():
        print(f'No Match: "{name}"')
        return name
    cleaned = LAWMAKER_NAME_CLEANING[name]
    return cleaned

def get_district(page):
    title = page.find('h3').text
    district = re.search(r'(SD|HD)\w+', title).group(0)
    district = district.replace('SD','SD ').replace('HD','HD ')
    return district

def get_address(page):
    address = page.find(string="Address").parent.text
    address = re.sub(r'\n\s+',' ', address)
    address = re.sub(r'\s+',' ', address)
    address = address.replace('Address ','')
    return address.strip(' ')

def get_phone(page):
    line = page.find(string=re.compile(r"Primary ph:"))
    phone = line.replace('Primary ph: ','')
    return phone
    
def get_email(page):
    line = page.find(string=re.compile(r"Email:"))
    email = line.replace('Email: ','').strip()
    return email
    

def get_city(page):
    address = page.find(string="Address").parent.text
    city = re.search(r'(?<=\n)(\w| )+(?=, MT)', address).group(0)
    return city.strip().title()

def get_sessions_served(page):
    table = page.find('strong', string='Legislative Service').parent.findNext('p')
    try:
        house_text = re.search('(?<=House:)\s+(.|\n)+?(?=<br/>|</p>)', str(table)).group(0)
    except AttributeError:
        house_text = ''
    house_session_links = BeautifulSoup(house_text, 'html.parser')

    try:
        senate_text = re.search('(?<=Senate:)\s+(.|\n)+?(?=<br/>|</p>)', str(table)).group(0)
    except AttributeError:
        senate_text = ''
    senate_session_links = BeautifulSoup(senate_text, 'html.parser')

    sessions = []
    for link in house_session_links.find_all(True):
        sessions.append({
            'year': link.text ,
            'url': f'https://leg.mt.gov/{link["href"]}',
            'chamber': 'house',
        })
    for link in senate_session_links.find_all(True):
        sessions.append({
            'year': link.text ,
            'url': f'https://leg.mt.gov/{link["href"]}',
            'chamber': 'senate',
        })
    return sessions

def get_committees(page_soup):
    assignment_table = page_soup.find(id='committeeAssignmentTable').find('tbody')
    rows = assignment_table.find_all('tr')
    
    committees = []
    for row in rows:
        cols = row.find_all('td')
        committee = re.search(r'\((S|H|J)\).+', cols[0].text).group(0)
        role = re.search(r'(Member|Chair|Vice Chair|Majority Vice Chair|Minority Vice Chair|NULL)', cols[1].text).group(0).replace('NULL','Member')
        committees.append({
            'committee': committee,
            'role': role,
        })
    return committees

def download_portrait(page, path):
    if os.path.exists(path):
        print('--Image already exists')
    else:
        portrait = page.find('img')
        src = portrait['src'].split(',')[1]
        im = Image.open(BytesIO(base64.b64decode(src)))
        im.save(path, 'PNG')
        print('--Image written to', path)

In [15]:
def get_roster(file):
    # Fetch list of names
    # HTML downloaded from https://leg.mt.gov/legislator-information/

    roster_html = codecs.open(file,'r').read()
    roster_soup = BeautifulSoup(roster_html, 'html.parser')
    rows = list(roster_soup.find('table', id='reports-table').find_all('tr'))
    headers = rows[0]
    rows = rows[1:]
    roster = []
    for row in rows:
        url = row.find('td', {"class":'nameCell'}).find('a')['href']
        name_text = row.find('td', {"class":'nameCell'}).text.strip().replace('\n                       ','').split(' \n                    \n')
        if len(name_text) > 1: note = name_text[1]
        else: note = ''
        #     email_url = row.find('td', {"class":'emailCell'}).find('a')['href']

        roster.append({
            'name': name_text[0],
            'party': row.find('td', {"class":'partyCell'}).text.strip().replace('\n                       ',''),
            'district': row.find('td', {"class":'seatCell'}).text.strip().replace('\n                    ',' '),
    #         'email': email_url,
            'phone': row.find('td', {"class":'phoneCell'}).text.strip().replace('\n                    ',', '),
            # 'url': f'https://leg.mt.gov/{url}',
            'url': url,
            'note': note,
        })
    return roster
    

def get_lawmaker(lawmaker, fetch_portraits=True, year='2023'):
    print('Fetching', lawmaker['name'], lawmaker['url'])
    session = requests.Session()
    r = session.get(lawmaker['url'])
    page = BeautifulSoup(r.text, 'html.parser')
    
    name = get_name(page)
    key = name.replace(' ','-')
    image_path = f'portraits/{year}/{key}.png'
    
    download_portrait(page, image_path) # Slow
    
    results = {
        'name': name,
        'district': get_district(page),
        'party': lawmaker['party'],
        'address': get_address(page),
        'city': get_city(page),
        'phone': get_phone(page),
        'email': get_email(page),
        'sessions': get_sessions_served(page),
        'committees': get_committees(page),
        'note': lawmaker['note'],
        'image_path': image_path,
        'source': lawmaker['url'],
    }
    return results

In [16]:
house_roster = get_roster('raw/web-roster-2023-house.html')
senate_roster = get_roster('raw/web-roster-2023-senate.html')
roster = house_roster + senate_roster

In [17]:
year = '2023'
details = []
for lawmaker in roster:
    # Temporary fix — skip Kathy Whitman w/ broken page
    if lawmaker['url'] != 'https://leg.mt.gov//legislator-information/roster/individual/6929':
        results = get_lawmaker(lawmaker, year=year)
        details.append(results)
    
with open(f'leg-roster-{year}.json', 'w') as f:
    stringed = json.dumps(details, indent=4)
    f.write(stringed)
    print('File written')

Fetching KIM
                    
                    ABBOTT https://leg.mt.gov/legislator-information/roster/individual/7467
No Match: "Kim Abbott"
--Image already exists
Fetching FRED
                    
                    ANDERSON https://leg.mt.gov/legislator-information/roster/individual/7617
No Match: "Fred Anderson"
--Image already exists
Fetching BRAD
                    
                    BARKER https://leg.mt.gov/legislator-information/roster/individual/7618
--Image already exists
Fetching DENISE
                    
                    BAUM https://leg.mt.gov/legislator-information/roster/individual/7619
--Image already exists
Fetching DAVID
                    
                    BEDEY https://leg.mt.gov/legislator-information/roster/individual/7473
No Match: "David Bedey"
--Image already exists
Fetching JAMES
                    
                    BERGSTROM https://leg.mt.gov/legislator-information/roster/individual/7620
--Image already exists
Fetching MARTA
       

In [1]:
# Manual additions

# Daniel Zolnikov service
zolnikov = {
    "sessions": [
        {
            "year": "2023",
            "url": "https://leg.mt.gov//legislator-information/roster/individual/7652",
            "chamber": "senate"
        },
        {
            "year": "2019",
            "url": "https://leg.mt.gov/legislator-information/roster/individual/5104",
            "chamber": "house"
        },
        {
            "year": "2017",
            "url": "https://leg.mt.gov/legislator-information/roster/individual/4787",
            "chamber": "house"
        },
        {
            "year": "2015",
            "url": "https://leg.mt.gov/legislator-information/roster/individual/4545",
            "chamber": "house"
        },
        {
            "year": "2013",
            "url": "https://leg.mt.gov/legislator-information/roster/individual/4158",
            "chamber": "house"
        }
    ]
}

### Zack Wirth
wirth = {
        "name": "Zack Wirth",
        "district": "HD 80",
        "party": "R",
        "address": "2020 CHEVALLIER DR WOLF CREEK, MT 59648-8747",
        "city": "Wolf Creek",
        "phone": "(406) 750-5852",
        "email": "Zack.Wirth@legmt.gov",
        "sessions": [
            {
                "year": "2023",
                "url": "https://leg.mt.gov/legislator-information/roster/individual/7654",
                "chamber": "house"
            }
        ],
        "committees": [
            {
                "committee": "(H) (H) Transportation",
                "role": "Member"
            },
            {
                "committee": "(H) (H) State Administration",
                "role": "Member"
            },
            {
                "committee": "(H) (H) Agriculture",
                "role": "Member"
            }
        ],
        "note": "",
        "image_path": "portraits/2023/Zack-Wirth.png",
        "source": "https://leg.mt.gov/legislator-information/roster/individual/7654"
    }